<a href="https://colab.research.google.com/github/hector6298/Deep-Learning-Collab-notebooks/blob/master/calibration_from_roadside_camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/zzh8829/yolov3-tf2
%cd yolov3-tf2/
#!pip install -r requirements-gpu.txt
!mkdir outframes

In [ ]:
!mkdir data
!cp "/content/drive/My Drive/YOLO_weights/yolov3.weights" "data/yolov3.weights"
!python convert.py


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 "https://github.com/tensorflow/models"

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from absl import app, flags, logging
from absl.flags import FLAGS
from IPython.display import clear_output
from skimage import feature, color, transform, io
from google.colab.patches import cv2_imshow
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from yolov3_tf2.models import (YoloV3, YoloV3Tiny)
from yolov3_tf2.dataset import transform_images
from yolov3_tf2.utils import draw_outputs
import cv2
import pandas as pd
import time
%matplotlib inline

In [ ]:
OUTPATH = None
CLASSES = ['car', 'truck', 'bus']
NCLASSES = len(CLASSES)
WEIGHTSPATH = "/content/yolov3-tf2/checkpoints/yolov3.tf"
YOLOSIZE = 416
TINY_YOLO = False
VIDEOPATH = "/content/drive/My Drive/speed_DS/center.avi"
app._run_init(['yolov3'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
def draw_outputs(img, outputs, class_names):
    boxes, objectness, classes, nums = outputs
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]
    wh = np.flip(img.shape[0:2])
    boxpoints = []
    for i in range(nums):
        x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
        x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
        boxpoints.append((x1y1[0], x1y1[1], x2y2[0], x2y2[1]))
        #img = cv2.rectangle(img, x1y1, x2y2, (255, 0, 0), 2)
        #img = cv2.putText(img, '{} {:.4f}'.format(
         ##  x1y1, cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
    return img, boxpoints

##Get car keypoint neural network

In [ ]:
!pip install --upgrade tensorflow-hub
import tensorflow_hub as hub

In [ ]:
MODEL_URL = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
def get_model(input_shape, output_nodes):
  inputs = tf.keras.layers.Input(shape=input_shape)
  base_model = hub.KerasLayer(MODEL_URL, trainable=True)
  regression_layer = tf.keras.layers.Dense(output_nodes, activation='tanh')

  x = inputs
  x = base_model(x)
  x = regression_layer(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/whole_keypointDNN112x112.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model.summary()

In [ ]:
front_csv = pd.read_csv('/content/drive/My Drive/car_measures/front_side2.csv')
back_csv = pd.read_csv('/content/drive/My Drive/car_measures/back_side2.csv')

##Get Object detection network

In [ ]:
yolo = YoloV3(classes=80)

yolo.load_weights(WEIGHTSPATH)
logging.info('weights loaded')

class_names = CLASSES
logging.info('classes loaded')

##Car crops flow

In [ ]:
times = []
vid = cv2.VideoCapture(VIDEOPATH)
out = None
fps = int(vid.get(cv2.CAP_PROP_FPS))

width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

detections = 300
frame = 0
flag = True


counter = 0


cars = np.empty((detections,112,112,3))
points_in_region = []
crops_points = []
crops_shapes = []
cars_org = []
sizes = []
while True:
    
    _, img = vid.read()

    if img is None:
        logging.warning("Empty Frame")
        time.sleep(0.1)
        continue
        
    if frame % 10 == 0:   
        img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_in = tf.expand_dims(img_in, 0)
        img_in = transform_images(img_in, YOLOSIZE)

        #t1 = time.time()
        boxes_, scores_, classes_, nums_ = yolo.predict(img_in)
        #print(nums.shape, scores.shape, classes.shape)
        
        img, boxPoints = draw_outputs(img, (boxes_, scores_, classes_, nums_), class_names)
        #objects, velocities = tracker.update(boxPoints, fps)

        for boxpoint in boxPoints:
          x1, y1, x2, y2 = boxpoint
          #CHECK ###########################################################################################################################
          centroid = (int((y1 + y2)/2.0), int((x1 + x2)/2.0))                                                                              #
          ##################################################################################################################################
          points_in_region.append(centroid)
          if counter < detections:
            crop = img[boxpoint[1]-25:boxpoint[3]+26, boxpoint[0]-25:boxpoint[2]+26, :]
            if crop.shape[0] > 112 and crop.shape[1] > 112:
              cars_org.append(crop)
              crops_shapes.append((crop.shape[0], crop.shape[1]))
              #CHECH #########################################################################################################################
              crops_points.append((y1,x1))                                                                                                   #
              ################################################################################################################################
              crop = cv2.resize(crop, (112,112))
              x = crop.astype('float32') / 255.0
              cars[counter] = x
              sizes.append([boxpoint[3]-boxpoint[1], boxpoint[2]-boxpoint[0]])
              counter += 1
          else:
            flag = False
            break
        if not flag:
          break
        print(counter)
    frame += 1


In [ ]:
plt.imshow(cars[2])

In [ ]:
predictions = np.empty((detections, 20))
batch_size = 64
steps_per_ds = int(detections / batch_size)
remainder_ds = detections % batch_size

for ind in range(steps_per_ds):
  preds = model(cars[(ind*batch_size):((ind+1)*batch_size)])
  predictions[(ind*batch_size):((ind+1)*batch_size)] = preds.numpy()

preds = model(cars[-remainder_ds:])
predictions[-remainder_ds:] = preds.numpy()


In [ ]:
sample_img1 = cars[2]
for i in range(0, 20, 2):
  #print(i, i+1)
  pt1 = predictions[2][i+1]*sample_img1.shape[0]
  pt1 = pt1.astype(int)
  pt2 = predictions[2][i]*sample_img1.shape[1]
  pt2 = pt2.astype(int)
  pt = (pt2, pt1)
  sample_img2 = cv2.circle(sample_img1, pt, 2, (255,0,0), 2) 
plt.imshow(sample_img2)
plt.show()
print(pt)

In [ ]:
def check_keypoint(csv,keypointInd, realPointInd):
  global keypoints
  global points
  global realPoints

  if keypoints[keypointInd] is not None:
    points.append(keypoints[keypointInd])
    realPoints[1].append((csv['x1'][realPointInd],csv['y1'][realPointInd],csv['z1'][realPointInd]))
    realPoints[2].append((csv['x2'][realPointInd],csv['y2'][realPointInd],csv['z2'][realPointInd]))
    realPoints[3].append((csv['x3'][realPointInd],csv['y3'][realPointInd],csv['z3'][realPointInd]))
    realPoints[4].append((csv['x4'][realPointInd],csv['y4'][realPointInd],csv['z4'][realPointInd]))
    realPoints[5].append((csv['x5'][realPointInd],csv['y5'][realPointInd],csv['z5'][realPointInd]))
  #else:
    #print("Nothing")


In [ ]:
 '''
'left front tire': 0,
'right front tire': 1,
'left back tire': 2,
'right back tire': 3,
'right headlight': 4,
'left headlight': 5,
'front plate': 6,
'left rear lamp': 7,
'right rear lamp': 8,
'rear plate': 9
'''

f = open('/content/points.txt',"w+")

for j in range(detections):

  points = []
  #counters = []
  realPoints = {1 : [],
                2 : [],
                3 : [],
                4 : [],
                5 : []}

  keypoints = { 0: None,
                1: None,
                2: None,
                3: None,
                4: None,
                5: None,
                6: None,
                7: None,
                8: None,
                9: None}
#crops_points 
#crops_shapes 

  prediction = predictions[j]
  counter = 0
  #print(f'working on detection {j}')
  for i in range(0, 20, 2):
    if prediction[i] >= 0 and prediction[i+1] >= 0:
      # CHECK #############################################################################################################
      pred_y = ((prediction[i+1]*crops_shapes[j][0]) + crops_points[j][0]) - height//2
      pred_x = (prediction[i]*crops_shapes[j][1] + crops_points[j][1]) - width//2
      ####################################################################################################################
      #keypoints[i//2] = (pred_y, pred_x)
      keypoints[i//2] = (pred_x, pred_y)
      counter += 1
  if counter >= 4:
    #counters.append(counter)
    #front
    if keypoints[4] is not None or keypoints[5] is not None:
      #front-right
      if keypoints[3] is not None or keypoints[1] is not None:
        check_keypoint(front_csv,4,1)
        check_keypoint(front_csv,5,0)
        check_keypoint(front_csv,3,6)
        check_keypoint(front_csv,1,4)
        check_keypoint(front_csv,6,2)
      #front-left
      elif keypoints[0] is not None or keypoints[2] is not None:
        check_keypoint(front_csv,4,1)
        check_keypoint(front_csv,5,0)
        check_keypoint(front_csv,0,3)
        check_keypoint(front_csv,2,5)
        check_keypoint(front_csv,6,2)

    #back
    elif keypoints[7] is not None or keypoints[8] is not None:
      #back-right
      if keypoints[3] is not None or keypoints[1] is not None:
        #search in back csv
        check_keypoint(back_csv,3,4)
        check_keypoint(back_csv,1,6)
        check_keypoint(back_csv,7,0)
        check_keypoint(back_csv,8,1)
        check_keypoint(back_csv,9,2)
      #back-left
      elif keypoints[0] is not None or keypoints[2] is not None:
        #seach in back csv
        check_keypoint(back_csv,0,5)
        check_keypoint(back_csv,2,3)
        check_keypoint(back_csv,7,0)
        check_keypoint(back_csv,8,1) 
        check_keypoint(back_csv,9,2)
        f.write(str(len(points))+"\n")

    f.write(str(len(points))+"\n")
    for i in range(len(points)):
      f.write(f"{points[i][0]} {points[i][1]}\n")
      f.write(f"{realPoints[1][i][0]} {realPoints[1][i][1]} {realPoints[1][i][2]}\n")
      f.write(f"{realPoints[2][i][0]} {realPoints[2][i][1]} {realPoints[2][i][2]}\n")
      f.write(f"{realPoints[3][i][0]} {realPoints[3][i][1]} {realPoints[3][i][2]}\n")
      f.write(f"{realPoints[4][i][0]} {realPoints[4][i][1]} {realPoints[4][i][2]}\n")
      f.write(f"{realPoints[5][i][0]} {realPoints[5][i][1]} {realPoints[5][i][2]}\n")

  else:
    print(f"Not enough points on instance {j}")
f.close()

In [ ]:
!cat /content/points.txt

## Get focal and camera pose algorithm

In [ ]:
!mkdir /content/p3pf
!git clone https://github.com/caomw/p3pf.git /content/p3pf

In [ ]:
%%writefile /content/p3pf/p3pf_example.cc
#include <Eigen/Dense>
#include <Eigen/StdVector>
#include <glog/logging.h>

#include <algorithm>
#include <chrono>
#include <fstream>
#include <random>

#include "camera_pose.h"
#include "eigen_helpers.h"
#include "p3pf_params.h"
#include "p3pf.h"
using namespace std;
int num_car_types = 5;
int main(int argc, char *argv[]) {
  gflags::ParseCommandLineFlags(&argc, &argv, true);
  google::InitGoogleLogging(argv[0]);

   ofstream outfile;
   outfile.open("/content/output.txt");
  // The original list of opening angles used for the experiments in the
  // paper.
  std::vector<double> opening_angles = { 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5,
      8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5,
      20.5, 21.5, 22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5, 30.5, 31.5,
      32.5, 33.5, 34.5, 35.5, 36.5, 37.5, 38.5, 39.5, 40.5, 41.5, 42.5, 43.5,
      44.5, 45.5, 46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5, 55.5,
      56.5, 57.5, 58.5, 59.5, 60.5, 61.5, 62.5, 63.5, 64.5, 65.5, 66.5, 67.5,
      68.5, 69.5, 70.5, 71.5, 72.5, 73.5, 74.5, 75.5, 76.5, 77.5, 78.5, 79.5,
      80.5, 81.5, 82.5, 83.5, 84.5, 85.5, 86.5, 87.5, 88.5, 89.5, 90.5, 91.5,
      92.5, 93.5, 94.5, 95.5, 96.5, 97.5, 98.5, 99.5};
  std::vector<double> prior_probabilities = { 0.0019519, 0.00060173, 0.00044518,
      0.0010176, 0.0048432, 0.0034343, 0.0077197, 0.0038012, 0.0034098,
      0.0062228, 0.0046328, 0.0039968, 0.0050633, 0.0044176, 0.0109, 0.0062325,
      0.0060466, 0.017308, 0.01993, 0.014945, 0.0065994, 0.0080915, 0.015024,
      0.012196, 0.015298, 0.011917, 0.014877, 0.011536, 0.0083802, 0.0090161,
      0.0080915, 0.010332, 0.011599, 0.0089868, 0.00839, 0.0093244, 0.0094515,
      0.0127, 0.0094613, 0.0087569, 0.010621, 0.010763, 0.012118, 0.014549,
      0.013189, 0.014353, 0.01563, 0.02813, 0.050389, 0.053666, 0.047938,
      0.065119, 0.065642, 0.02904, 0.016261, 0.011364, 0.0088645, 0.0071718,
      0.006394, 0.0055574, 0.0067609, 0.010704, 0.022083, 0.025263, 0.015508,
      0.021892, 0.02358, 0.0093928, 0.0039528, 0.0026564, 0.0019813, 0.0021476,
      0.0025586, 0.0023384, 0.001179, 0.0011301, 0.0010322, 0.00092461,
      0.00077295, 0.00072892, 0.00075338, 0.00076806, 0.00068, 0.00089526,
      0.00086101, 0.00082187, 0.00082187, 0.00088058, 0.00091972, 0.00089036,
      0.00079252, 0.00076806, 0.00090015, 0.00088058, 0.0015214, 0.0020987,
      0.0016829, 0.0012328, 0.00091972, 0.00061641 };
  std::ifstream infile("/content/points.txt");
  while (!infile.eof()){
    // Initializes the parameters of the p3pf algorithm.
    p3pf::P3PfParameters p3pf_params;
    p3pf_params.ransac_parameters_.failure_probability = 0.01;
    p3pf_params.ransac_parameters_.min_inlier_ratio = 0.1;
    p3pf_params.ransac_parameters_.squared_inlier_threshold = 10.0;
    p3pf_params.ransac_parameters_.random_seed = 0;
    p3pf_params.ransac_parameters_.use_T_1_1_test = true;


    p3pf_params.cdf_inlier_ratios_ = {0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
      0.8, 0.9, 1.0};


    int points;
    double x2d, y2d;
    double x3d, y3d, z3d;
    std::vector<Eigen::Vector2d> points2D;
    std::vector<Eigen::Vector3d> *points3D = new std::vector<Eigen::Vector3d>[num_car_types];
    
    
    if(  infile >> points ){
    
      for(int i = 0; i < points; i++){
        infile >> x2d >> y2d;
        points2D.push_back({x2d,y2d});

        for(int j = 0; j < num_car_types; j++){
          infile >> x3d >> y3d >> z3d;
          points3D[j].push_back({x3d,y3d,z3d});
        }
      }
    }

    // Converts the opening angles used for sampling to focal length values.
    p3pf_params.focal_length_values_ = opening_angles;
    for (size_t i = 0; i < opening_angles.size(); ++i) {
      p3pf_params.focal_length_values_[i] = 320.0
          / tan(opening_angles[i] / 2.0 * M_PI / 180.0);
    }

    p3pf_params.prior_probabilities_ = prior_probabilities;

    for(int i = 0; i < num_car_types; i++){
      // Initializes and runs P3P(f).
      p3pf::P3Pf p3pf;
      p3pf.Init(p3pf_params);
      p3pf::P3PfResult result;
      std::chrono::high_resolution_clock::time_point t1 =
          std::chrono::high_resolution_clock::now();
      p3pf.ComputePose(points2D, points3D[i], &result);
      std::chrono::high_resolution_clock::time_point t2 =
          std::chrono::high_resolution_clock::now();
      std::chrono::duration<double> run_time = std::chrono::duration_cast<
          std::chrono::duration<double>>(t2 - t1);

      /*"""
      outfile << "Found number of inliers: " << result.num_inliers_ << endl;

      outfile << " Estimated "
                << "focal length: " << result.pose_.focal_length() << endl;
      """*/
      Eigen::Vector3d t;
      Eigen::Matrix3d R;
      result.pose_.rotation_matrix(&R);
      result.pose_.translation(&t);
      Eigen::Vector3d c;
      c = -R.transpose() * t;

      /*"""
      outfile << "Computed camera center: "
                << c.transpose() << endl;
      outfile << "P3P(f) took " << result.num_generated_random_samples_
                << " iterations to compute the pose" << endl;
      outfile << "Running P3P(f) took " << run_time.count() << " seconds " << endl;
      """*/
      outfile << result.pose_.focal_length() << endl;
      outfile << R(0,0) << " " << R(0,1) << " " << R(0,2) << " " << t(0) << endl;
      outfile << R(1,0) << " " << R(1,1) << " " << R(1,2) << " " << t(1) << endl;
      outfile << R(2,0) << " " << R(2,1) << " " << R(2,2) << " " << t(2) << endl;


    }
  }
  return 0;
}


In [ ]:
!sudo apt-get install -y scons
!sudo apt-get install libeigen3-dev
!sudo apt-get install -y libgoogle-glog-dev
%cd /content/p3pf
!scons


##Compute and retrieve results

In [ ]:
!./p3pf_example

In [ ]:
!cat /content/output.txt

##Filtering calibrations

Get calibrations into python

In [ ]:
focals = []
pose_mats = []
mat = []
with open("/content/output.txt","r") as calib_file:
  contents = calib_file.readlines()
  for i in range(len(contents)):
    if i % 4 == 0:
      focals.append(float(contents[i]))
      if i != 0:
        pose_mats.append(mat)
      mat = []
    else:
      row = list(map(float,contents[i].split(' ')))
      mat.append(row)
pose_mats = np.array(pose_mats)

Road regions of interest

In [ ]:
points_in_region = np.array(points_in_region)
my, mx = np.mean(points_in_region[:,0]), np.mean(points_in_region[:,1])
point_of_interest = (mx,my)

Formulas

In [ ]:
from numpy import linalg as LA
def OrientationFilter(pose_mats, threshold):
  
  """
  Filter out outlier pose mats based on orientation of the z axis.
  pose_mats: numpy ndarray of shape [None,3,4]. A batch of pose matrices P = [R|T]
  threshold: Int that represents the percentage of pose matrices to keep
  """

  assert len(pose_mats.shape) == 3 and pose_mats[0].shape == (3,4), f"Shape is invalid, expected [None,3,3] got: {pose_mats.shape}"
  arccos_arr = np.array([])
  #Take third column of pose matrix i.e. the Z axis of rotation
  z = pose_mats[:,:,2]
  zavg = np.mean(z, axis=0)
  zavg_norm = LA.norm(zavg)
  mapping_dict = dict()
  for i in range(len(z)):
    z_ = np.inner(z[i], zavg)
    norms = LA.norm(z[i]) * zavg_norm
    quotient = z_/norms
    arccos_val = np.arccos(quotient)
    arccos_arr = np.append(arccos_arr, arccos_val)
    mapping_dict[arccos_val] = i
  sorted_arccos = np.sort(arccos_arr)
  sorted_arccos = sorted_arccos[:int((len(sorted_arccos)/100)*threshold)]
  new_mats = []
  for arccos in sorted_arccos:
    new_mats.append(pose_mats[mapping_dict[arccos]])
  new_mats = np.array(new_mats)
  assert len(new_mats.shape) == 3, f"Got invalid shape for output matrices, got: {new_mats.shape}"
  return  new_mats


In [ ]:
def reproject2Dto3D(point2D, focal_length, pose_mat, Z = 0):
  """
  Reproject a image point into world points
  point2D: tuple or array of 2 integers. Image coordinates [y,x]
  focal_length: float. Camera focal length
  pose_mat: numpy ndarray of shape [3,3]. The pose matrix P = [R|T]
  Z: Known value of height from the road up in world points measures.

  returns tuple of 3 integers. The world points
  """

  assert pose_mat.shape == (3,4), f"pose_mat should be of shape [3,3] got: {pose_mat.shape}"
  assert focal_length > 0, "Expected positive focal_length value"
  assert Z >= 0, "Expected positive or zero Z value"

  Xnum = (focal_length*pose_mat[0][1] - pose_mat[2][1]*point2D[0])*(Z*pose_mat[1][2]+pose_mat[1][3]) - (focal_length*pose_mat[1][1] - pose_mat[2][1]*point2D[1])*(Z*pose_mat[0][2]+pose_mat[0][0]) - (Z*pose_mat[2][2]+pose_mat[2][3])*(pose_mat[0][1]*point2D[1]-pose_mat[1][1]*point2D[0])
  Xden = focal_length*pose_mat[0][0]*pose_mat[1][1] - focal_length*pose_mat[0][1]*pose_mat[1][0] - pose_mat[0][0]*pose_mat[2][1]*point2D[1] + pose_mat[0][1]*pose_mat[2][0]*point2D[1] + pose_mat[1][0]*pose_mat[2][1]*point2D[0] - pose_mat[1][1]*pose_mat[2][0]*point2D[0]
  Ynum = -(focal_length*pose_mat[0][0]-pose_mat[2][0]*point2D[0])*(Z*pose_mat[1][2]+pose_mat[1][3]) + (focal_length*pose_mat[1][0]-pose_mat[2][0]*point2D[1])*(Z*pose_mat[0][2]+pose_mat[0][3]) + (Z*pose_mat[2][2]+pose_mat[2][3])*(pose_mat[0][0]*point2D[1]-pose_mat[1][0]*point2D[0])
  X = Xnum/Xden
  Y = Ynum/Xden
  assert np.array([X,Y,Z]).shape == (3,), f"Error: world point not getting expected shape, got {np.array([X,Y,Z]).shape}"
  return (X,Y,Z)


In [ ]:
def distance2Camera(point3D, pose_mat):
  """
  Compute distance from road to camera using euclidean distance.
  point3D: tuple or array of 3 integers. World coordinates [X,Y,Z]
  pose_mat: numpy ndarray of shape [3,3]. The pose matrix P = [R|T]
  """
  
  return np.sqrt((point3D[0]-pose_mat[0][3])**2 + (point3D[1]-pose_mat[1][3])**2 + (point3D[2]-pose_mat[2][3])**2)

In [ ]:
def displacementFilter(pose_mats,p, focal, threshold):

  """
  Filters pose matrices based on displacement produced from the same image point reprojected to world points for each pose matrix
  pose_mats: numpy ndarray of shape [None,3,4]. A batch of pose matrices P = [R|T]
  focal: float. Camera focal length
  p: tuple of two ints. Image point corresponding to center of region of interest (the road)
  threshold: Int that represents the percentage of pose matrices to keep
  """

  assert len(pose_mats.shape) == 3 and pose_mats[0].shape == (3,4), f"Shape is invalid, expected [None,3,3] got: {pose_mats.shape}"
  assert focal > 0, "Expected positive focal_length value"
  assert threshold > 0 and threshold <= 100, f"Expected a threshold value from 0 to 100, got: {threshold}"

  displacements_container = []
  mapping_dict = dict()
  new_mats = []
  i = 0
  for pose_mat in pose_mats:
    reprojected_center = reproject2Dto3D(p,focal,pose_mat)
    distance = distance2Camera(reprojected_center, pose_mat)
    displacements_container.append(distance)
    mapping_dict[distance] = i
    i += 1
  displacements_container.sort()
  for i in range(int(len(displacements_container)*threshold/100)):
    new_mats.append(pose_mats[mapping_dict[displacements_container[i]]])
  new_mats = np.array(new_mats)
  assert len(new_mats.shape) == 3, f"Got invalid shape for output matrices, got: {new_mats.shape}"
  return new_mats



In [ ]:
def median_displacement_posemat(pose_mats,p, focal):
  assert len(pose_mats.shape) == 3 and pose_mats[0].shape == (3,4), f"Shape is invalid, expected [None,3,3] got: {pose_mats.shape}"
  assert focal > 0, "Expected positive focal_length value"

  """
  Similar fashion from displacement_filer, but pick median of the pose matrices with respect of their reprojection displacement
  pose_mats: numpy ndarray of shape [None,3,4]. A batch of pose matrices P = [R|T]
  focal: float. Camera focal length
  p: tuple of two ints. Image point corresponding to center of region of interest (the road)
  """

  displacements_container = []
  mapping_dict = dict()
  i = 0
  for pose_mat in pose_mats:
    reprojected_center = reproject2Dto3D(p,focal,pose_mat)
    distance = distance2Camera(reprojected_center, pose_mat)
    displacements_container.append(distance)
    mapping_dict[distance] = i
    i += 1
  displacements_container.sort()
  median = len(displacements_container)//2
  
  return pose_mats[mapping_dict[displacements_container[median]]]



In [ ]:
def mat2axis_angle(rotation_mat, epsilon=0.01, epsilon2=0.1):
  assert rotation_mat.shape == [3,3], "not valid rotation matrix"
  if (abs(rotation[0][1]-rotation[1][0]) < epsilon)  and (abs(rotation[0][2]-rotation[2][0]) < epsilon) and (abs(rotation[1][2]-rotation[2][1])<epsilon):
    if (abs(rotation[0][1]+rotation[1][0]) < epsilon2) and (abs(rotation[0][2]+rotation[2][0]) < epsilon2) and (abs(rotation[1][2]+rotation[2][1]) < epsilon2) and (abs(rotation[0][0]+rotation[1][1]+rotation[2][2]-3) < epsilon2):
      return (0,1,0,0)

    angle = np.pi
    xx = (rotation[0][0]+1)/2
    yy = (rotation[1][1]+1)/2
    zz = (rotation[2][2]+1)/2
    xy = (rotation[0][1]+rotation[1][0])/4
    xz = (rotation[0][2]+rotation[2][0])/4
    yz = (rotation[1][2]+rotation[2][1])/4

    if (xx > yy) and (xx > zz):
      if xx < epsilon:
        x = 0
        y = 0.7071
        z = 0.7071
      else:
        x = math.sqrt(xx)
        y = xy/x
        z = xz/x
    elif yy > zz:
      if yy < epsilon:
        x = 0.7071
        y = 0
        z = 0.7071
      else:
        x = xy/y
        y = math.sqrt(yy)
        z = yz/y
    else:
      if zz < epsilon:
        x = 0.7071
        y = 0.7071
        z = 0
      else:
        x = xz/z
        y = yz/z
        z = math.sqrt(zz)
    
    return (angle,x,y,z)
  s = math.sqrt( (rotation[2][1]-rotaion[1][2])*(rotation[2][1]-rotation[1][2])+(rotation[0][2]-rotation[2][0])*(rotation[0][2]-rotation[2][0])+(rotation[1][0]-rotation[0][1])*(rotation[1][0]-rotation[0][1]))
  if abs(s) < 0.001:
    s = 1
    print("warning matrix is not orthogonal")
  angle = math.acos((rotation[0][0]+rotation[1][1]+rotation[2][2]-1)/2)
  x = (rotation[2][1]-rotation[1][2])/s
  y = (rotation[0][2]-rotation[2][0])/s
  z = (rotation[1][0]-rotation[0][1])/s
  return (angle,x,y,z)




In [ ]:
def axis_angle2matrix(axis, angle):
  c = np.cos(angle)
  s = np.sin(angle)
  t = 1.0 - c
  mag = LA.norm(axis)
  if mag == 0:
    raise "Error, 0 magnitude"
  else:
    axis = axis/mag
  
  m00 = c + pow(axis[0],2)*t
  m11 = c + pow(axis[1],2)*t
  m22 = c + pow(axis[2],2)*t

  temp1 = axis[0]*axis[1]*t
  temp2 = axis[2]*s
  m10 =  temp1 + temp2
  m01 = temp1 - temp2

  temp1 = axis[0]*axis[2]*t
  temp2 = axis[1]*s
  m20 = temp1 - temp2
  m02 = temp1 + temp2

  temp1 = axis[1]*axis[2]*t
  temp2 = axis[0]*s
  m21 = temp1 + temp2
  m12 = temp1 - temp2

  rotation = [[m00,m01,m02],
              [m10,m11,m12],
              [m20,m21,m22]]
  return rotation

In [ ]:
def average_rotations(rotations, epsilon, maxit=1000):
  R = rotations[0]
  max = len(rotations)
  r = [0,0,0]
  mean = [0,0,0,0]
  angle = 0
  while maxit < 1000:
    for i in range(1,len(rotations)):
      RTR = np.matmul(R,rotations[i])
      axis_angle = mat2axis_angle(RTR)
      log_vector = axis_angle[0]*[axis_angle[1],axis_angle[2],axis_angle[3]]
      mean += axis_angle
      r = r + log_vector
    r = r/len(rotations)
    mean = mean/len(rotations)
    if LA.norm(r) < epsilon
      return R
    R_ = axis_angle2matrix(mean[1],mean[2],mean[3],mean[0]) 
    R = np.matmul(R,R_)
  return R

In [ ]:
from scipy.spatial.transform import Rotation
def filterCalibrations(pose_mats, focal, p_org,frame_height, frame_width):
  """
  Method for projection matrix filtration inspired by AutoCalib
  pose_mats: numpy ndarray of shape [None,3,4]. A batch of pose matrices P = [R|T]
  focal: float. Camera focal length
  p: tuple of two ints. Image point corresponding to center of region of interest (the road)
  """
  assert len(pose_mats.shape) == 3 and pose_mats[0].shape == (3,4), f"Shape is invalid, expected [None,3,3] got: {pose_mats.shape}"
  assert focal > 0, "Expected positive focal_length value"
  assert p_org[0] >= 0 and p_org[1] >=0, f"Expected point p with positive or zero coordinates, got: {p}"
  p = [0,0]
  p[0] = p_org[0] - frame_width/2
  p[1] = p_org[1] - frame_height/2
  new_mats = OrientationFilter(pose_mats,75)
  new_mats = displacementFilter(new_mats, p, focal, 50)
  new_mats = OrientationFilter(new_mats, 75)
  rotations = new_mats[:,:,:-1]
  #a vg_rotation = average_rotations(rotations,0.01)
  rotations = Rotation.from_matrix(rotations)
  mean_rotation = rotations.mean().as_matrix()
  for i in range(len(new_mats)):
    new_mats[i,:,:-1] = mean_rotation
  estimate = median_displacement_posemat(pose_mats,p, focal)

  return estimate


In [ ]:
def medianFocalLength(focal_length_array):
  """
  Compute median of focal_lengths.
  focal_length_array: Python tuple or list of floats. The array of focal length values.
  """
  focal_length_array.sort()
  length = len(focal_length_array)
  if length %2:
    final_focal = (focal_length_array[(length//2)+1] + focal_length_array[length//2])/2.
  else:
    final_focal = focal_length_array[length//2]
  return final_focal

Filtering!!

In [ ]:
final_focal_length = medianFocalLength(focals)
print(final_focal_length)

In [ ]:
final_pose = filterCalibrations(pose_mats,final_focal_length,point_of_interest, height, width)
print(final_pose)